In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional,GRU, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, MaxPool1D,BatchNormalization,Activation,Flatten
from keras.models import Model

# from keras.preprocessing import sequence
from keras.datasets import imdb
from matplotlib import pyplot as plt
import pandas as pd

from keras import backend as K
from keras.engine.topology import Layer

In [ ]:
class Self_Attention(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Self_Attention, self).__init__(**kwargs)

    def build(self, input_shape):

        #inputs.shape = (batch_size, time_steps, seq_len)
        self.kernel = self.add_weight(name='kernel',
                                      shape=(3,input_shape[2], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)

        super(Self_Attention, self).build(input_shape)  

    def call(self, x):
        WQ = K.dot(x, self.kernel[0])
        WK = K.dot(x, self.kernel[1])
        WV = K.dot(x, self.kernel[2])

        print("WQ.shape",WQ.shape)

        print("K.permute_dimensions(WK, [0, 2, 1]).shape",K.permute_dimensions(WK, [0, 2, 1]).shape)


        QK = K.batch_dot(WQ,K.permute_dimensions(WK, [0, 2, 1]))

        QK = QK / (64**0.5)

        QK = K.softmax(QK)

        print("QK.shape",QK.shape)

        V = K.batch_dot(QK,WV)

        return V

    def compute_output_shape(self, input_shape):

        return (input_shape[0],input_shape[1],self.output_dim)

In [ ]:
EMBEDDING_FILE = 'glove.840B.300d.txt'
train_x = pd.read_csv('data/2018train.csv')
test_x = pd.read_csv('data/2018dev2.csv')

max_features = 16803
maxlen = 150
embed_size = 300

In [ ]:
train_x['Tweet'].fillna(' ')
test_x['Tweet'].fillna(' ')
train_y = train_x[
    ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].values
train_x = train_x.astype(str)
train_x = train_x['Tweet'].str.lower()

test_x = test_x['Tweet'].str.lower()
# test_y = test_x[
#     ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].values

# Vectorize text + Prepare GloVe Embedding
tokenizer = text.Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

train_x = sequence.pad_sequences(train_x, maxlen=maxlen)
test_x = sequence.pad_sequences(test_x, maxlen=maxlen)

In [ ]:
embeddings_index = {}
with open(EMBEDDING_FILE, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Build Model
inp = Input(shape=(maxlen,))
# S_inputs = Input(shape=(64,), dtype='int32')

# embeddings = Embedding(max_features, 128)(S_inputs)


# O_seq = Self_Attention(128)(embeddings)
embeddings = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = Self_Attention(128)(embeddings)
# x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = Bidirectional(GRU(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(11, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prediction
batch_size = 32
epochs = 10

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)
predictions = model.predict(test_x, batch_size=batch_size, verbose=1)

WQ.shape (None, 150, 128)
K.permute_dimensions(WK, [0, 2, 1]).shape (None, 128, 150)
QK.shape (None, 150, 150)
Epoch 1/10
WQ.shape (None, 150, 128)
K.permute_dimensions(WK, [0, 2, 1]).shape (None, 128, 150)
QK.shape (None, 150, 150)
WQ.shape (None, 150, 128)
K.permute_dimensions(WK, [0, 2, 1]).shape (None, 128, 150)
QK.shape (None, 150, 150)
213/213 [==============================] - 241s 1s/step - loss: 0.4800 - accuracy: 0.2574
Epoch 2/10
213/213 [==============================] - 234s 1s/step - loss: 0.3301 - accuracy: 0.4710
Epoch 3/10
213/213 [==============================] - 229s 1s/step - loss: 0.2858 - accuracy: 0.5073
Epoch 4/10
213/213 [==============================] - 230s 1s/step - loss: 0.2513 - accuracy: 0.5346
Epoch 5/10
213/213 [==============================] - 231s 1s/step - loss: 0.2202 - accuracy: 0.5645
Epoch 6/10
213/213 [==============================] - 230s 1s/step - loss: 0.1964 - accuracy: 0.5206
Epoch 7/10
213/213 [==============================] - 229s 1s

In [ ]:
# Build Model
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(11, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prediction
batch_size = 32
epochs = 10

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)
predictions = model.predict(test_x, batch_size=batch_size, verbose=1)

Epoch 1/10
213/213 [==============================] - 260s 1s/step - loss: 0.4600 - accuracy: 0.2788
Epoch 2/10
213/213 [==============================] - 249s 1s/step - loss: 0.3294 - accuracy: 0.4843
Epoch 3/10
213/213 [==============================] - 246s 1s/step - loss: 0.2901 - accuracy: 0.5222
Epoch 4/10
213/213 [==============================] - 244s 1s/step - loss: 0.2569 - accuracy: 0.5294
Epoch 5/10
213/213 [==============================] - 243s 1s/step - loss: 0.2328 - accuracy: 0.5242
Epoch 6/10
213/213 [==============================] - 241s 1s/step - loss: 0.2017 - accuracy: 0.5609
Epoch 7/10
213/213 [==============================] - 245s 1s/step - loss: 0.1760 - accuracy: 0.5675
Epoch 8/10
213/213 [==============================] - 244s 1s/step - loss: 0.1507 - accuracy: 0.5502
Epoch 9/10
213/213 [==============================] - 245s 1s/step - loss: 0.1294 - accuracy: 0.5624
Epoch 10/10
28/28 [==============================] - 3s 79ms/step


In [ ]:
# Build Model
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = Bidirectional(GRU(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(11, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prediction
batch_size = 32
epochs = 10

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)
predictions = model.predict(test_x, batch_size=batch_size, verbose=1)

Epoch 1/10
213/213 [==============================] - 239s 1s/step - loss: 0.4609 - accuracy: 0.3043
Epoch 2/10
213/213 [==============================] - 234s 1s/step - loss: 0.3285 - accuracy: 0.4927
Epoch 3/10
213/213 [==============================] - 223s 1s/step - loss: 0.2840 - accuracy: 0.5426
Epoch 4/10
213/213 [==============================] - 221s 1s/step - loss: 0.2587 - accuracy: 0.5491
Epoch 5/10
213/213 [==============================] - 220s 1s/step - loss: 0.2325 - accuracy: 0.5505
Epoch 6/10
213/213 [==============================] - 220s 1s/step - loss: 0.2053 - accuracy: 0.5608
Epoch 7/10
213/213 [==============================] - 222s 1s/step - loss: 0.1739 - accuracy: 0.5645
Epoch 8/10
213/213 [==============================] - 222s 1s/step - loss: 0.1512 - accuracy: 0.5495
Epoch 9/10
213/213 [==============================] - 221s 1s/step - loss: 0.1303 - accuracy: 0.5586
Epoch 10/10
28/28 [==============================] - 2s 73ms/step


In [ ]:
# Build Model
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15)(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(11, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prediction
batch_size = 32
epochs = 10

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)
predictions = model.predict(test_x, batch_size=batch_size, verbose=1)

Epoch 1/10
213/213 [==============================] - 127s 586ms/step - loss: 0.4798 - accuracy: 0.2384
Epoch 2/10
213/213 [==============================] - 125s 586ms/step - loss: 0.3495 - accuracy: 0.4023
Epoch 3/10
213/213 [==============================] - 124s 584ms/step - loss: 0.3076 - accuracy: 0.5219
Epoch 4/10
213/213 [==============================] - 124s 582ms/step - loss: 0.2798 - accuracy: 0.5371
Epoch 5/10
213/213 [==============================] - 124s 584ms/step - loss: 0.2434 - accuracy: 0.5553
Epoch 6/10
213/213 [==============================] - 124s 584ms/step - loss: 0.2155 - accuracy: 0.5815
Epoch 7/10
213/213 [==============================] - 125s 585ms/step - loss: 0.1907 - accuracy: 0.5584
Epoch 8/10
213/213 [==============================] - 125s 587ms/step - loss: 0.1720 - accuracy: 0.5883
Epoch 9/10
213/213 [==============================] - 131s 616ms/step - loss: 0.1576 - accuracy: 0.5693
Epoch 10/10
28/28 [==============================] - 1s 43ms/ste

In [ ]:
# Build Model
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = GRU(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15)(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(11, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prediction
batch_size = 32
epochs = 10

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)
predictions = model.predict(test_x, batch_size=batch_size, verbose=1)

Epoch 1/10
213/213 [==============================] - 119s 550ms/step - loss: 0.4713 - accuracy: 0.3029
Epoch 2/10
213/213 [==============================] - 117s 547ms/step - loss: 0.3407 - accuracy: 0.5087
Epoch 3/10
213/213 [==============================] - 115s 539ms/step - loss: 0.3003 - accuracy: 0.5161
Epoch 4/10
213/213 [==============================] - 116s 546ms/step - loss: 0.2725 - accuracy: 0.5030
Epoch 5/10
213/213 [==============================] - 116s 546ms/step - loss: 0.2478 - accuracy: 0.5443
Epoch 6/10
213/213 [==============================] - 115s 539ms/step - loss: 0.2194 - accuracy: 0.5453
Epoch 7/10
213/213 [==============================] - 115s 539ms/step - loss: 0.1931 - accuracy: 0.5542
Epoch 8/10
213/213 [==============================] - 115s 542ms/step - loss: 0.1754 - accuracy: 0.5429
Epoch 9/10
213/213 [==============================] - 114s 536ms/step - loss: 0.1476 - accuracy: 0.5561
Epoch 10/10
28/28 [==============================] - 1s 38ms/ste

In [ ]:
x_input2 = Input(shape=(maxlen,))
emb1 = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(x_input2)
pool_output = []
kernel_sizes = [2, 3, 4,5]
for kernel_size in kernel_sizes:
    c = Conv1D(filters=64, kernel_size=kernel_size, strides=1)(emb1)
    p = MaxPool1D(pool_size=int(c.shape[1]))(c)
    pool_output.append(p)
pool_output = concatenate([p for p in pool_output])
normal = BatchNormalization()(pool_output)
act = Activation("relu")(normal)
flatten = Flatten()(act)
dropout = Dropout(0.7)(flatten)
output = Dense(11, activation='softmax')(dropout)
model = Model(inputs = x_input2, outputs=output)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prediction
batch_size = 32
epochs = 10

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)
predictions = model.predict(test_x, batch_size=batch_size, verbose=1)

Epoch 1/10
213/213 [==============================] - 13s 55ms/step - loss: 0.6717 - accuracy: 0.1905
Epoch 2/10
213/213 [==============================] - 12s 54ms/step - loss: 0.4187 - accuracy: 0.3658
Epoch 3/10
213/213 [==============================] - 12s 55ms/step - loss: 0.3562 - accuracy: 0.4384
Epoch 4/10
213/213 [==============================] - 12s 55ms/step - loss: 0.3202 - accuracy: 0.4833
Epoch 5/10
213/213 [==============================] - 11s 54ms/step - loss: 0.2976 - accuracy: 0.4854
Epoch 6/10
213/213 [==============================] - 12s 54ms/step - loss: 0.2799 - accuracy: 0.5197
Epoch 7/10
213/213 [==============================] - 12s 54ms/step - loss: 0.2642 - accuracy: 0.4993
Epoch 8/10
213/213 [==============================] - 12s 54ms/step - loss: 0.2515 - accuracy: 0.5316
Epoch 9/10
213/213 [==============================] - 11s 54ms/step - loss: 0.2423 - accuracy: 0.5170
Epoch 10/10
28/28 [==============================] - 0s 5ms/step


In [ ]:
submission = pd.read_csv('data/2018dev2.csv')
submission[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness',
            'surprise', 'trust']] = predictions
submission.to_csv('output/2018dev112.csv', index=False)